# Segmenting and Clustering Neighborhoods in Toronto

## Toronto - Postal Codes

## Get the wiki page which contains postal codes of 

In [1]:
import requests

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wiki_page = requests.get(url)

## Parse html content

In [2]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(wiki_page.content, 'html.parser')

# soup.children[0] -> <class 'bs4.element.Doctype'>
# soup.children[1] -> <class 'bs4.element.NavigableString'>
# soup.children[2] -> <html>...</html>
html = list(soup.children)[2]

## Extract header information

In [3]:
print(soup.find('title').get_text())

List of postal codes of Canada: M - Wikipedia


## Extract portal codes

In [4]:
# There are 3 tables on the wiki page, however, we only need the first one
table = soup.find('table')
cells = table.findAll('td')
print('Number of cells: ', len(cells))

Number of cells:  180


### Function for parsing content in each cell

In [5]:
import re

pattern = re.compile(r"(?P<code>M\d[A-Z])(?P<borough>[\w\d\s\']+)(?P<nbs>.+)")

def parse(text):
    matches = pattern.search(text)
    code = matches.group('code')
    borough = matches.group('borough')
    nbs = matches.group('nbs')
    if ((borough is not None) and (nbs is None)):
        nbs = borough
    else:
        nbs = re.sub(r'\s+|\)', ' ', nbs)
        nbs = re.sub(r'[\(/]', ', ', nbs)
        nbs = re.sub(r'(^,|,$)', '', nbs)
        nbs = re.sub(r'\s+', ' ', nbs)
    return (code, borough, nbs)

### Loop through all cells, and parse their contents

In [6]:
import pandas as pd

NEG_KEY = 'Not assigned'

pc_dict = {}
pc_dict['PostalCode'] = []
pc_dict['Borough'] = []
pc_dict['Neighborhood'] = []

for cell in cells:
    content = cell.get_text()
    if NEG_KEY in content:
        continue

    code, borough, nbs = parse(content)
    
    pc_dict['PostalCode'].append(code)
    pc_dict['Borough'].append(borough)
    pc_dict['Neighborhood'].append(nbs)

ds = pd.DataFrame(pc_dict, columns=['PostalCode', 'Borough', 'Neighborhood'])
ds.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


### Clean-up/Patch

In [7]:
print('Exceptions:')
print(ds[ds['PostalCode'].isin(['M5W', 'M7R', 'M7Y', 'M9W'])])

ds.loc[ds['PostalCode'] == 'M7Y', 'Borough'] = 'East Toronto'
ds.loc[ds['PostalCode'] == 'M7Y', 'Neighborhood'] = 'Business Reply Mail Processing Centre 969 Eastern'

ds.loc[ds['PostalCode'] == 'M7R', 'Borough'] = 'Mississauga'
ds.loc[ds['PostalCode'] == 'M7R', 'Neighborhood'] = 'Canada Post Gateway Processing Centre'

ds.loc[ds['PostalCode'] == 'M5W', 'Borough'] = 'Downtown Toronto'
ds.loc[ds['PostalCode'] == 'M5W', 'Neighborhood'] = 'Stn A PO Boxes 25 The Esplanade'

ds.loc[ds['PostalCode'] == 'M9W', 'Borough'] = 'Etobicoke Northwest'

Exceptions:
    PostalCode                                            Borough  \
76         M7R   MississaugaCanada Post Gateway Processing Centre   
92         M5W     Downtown TorontoStn A PO Boxes25 The Esplanade   
94         M9W                                 EtobicokeNorthwest   
100        M7Y  East TorontoBusiness reply mail Processing Cen...   

                                          Neighborhood  
76                                     Enclave of L4W   
92                                     Enclave of M5E   
94    Clairville , Humberwood , Woodbine Downs , We...  
100                                    Enclave of M4L   


In [8]:
print('After patching')
print(ds[ds['PostalCode'].isin(['M5W', 'M7R', 'M7Y', 'M9W'])])

After patching
    PostalCode              Borough  \
76         M7R          Mississauga   
92         M5W     Downtown Toronto   
94         M9W  Etobicoke Northwest   
100        M7Y         East Toronto   

                                          Neighborhood  
76               Canada Post Gateway Processing Centre  
92                     Stn A PO Boxes 25 The Esplanade  
94    Clairville , Humberwood , Woodbine Downs , We...  
100  Business Reply Mail Processing Centre 969 Eastern  


## Summary

In [9]:
print('Number of rows: ', ds.shape[0])

Number of rows:  103
